In [2]:
import os
import asyncio
import pandas as pd
import pandas_datareader.data as pdr

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

import itertools
import pmdarima as pm
import time
import sys
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import datetime, timedelta

# 현재 날짜 가져오기
today = datetime.now() 

# 날짜를 YYYYMMDD 형식으로 변환
today_str = today.strftime('%Y-%m-%d')
one_year_ago = today - timedelta(days=365)
one_year_ago_str = one_year_ago.strftime('%Y%m%d')
print(today_str,one_year_ago_str)

2024-06-21 20230622


In [51]:
from mysqlconnect import connect_to_mysql

connection = connect_to_mysql()

cursor = connection.cursor()

hitit-user
Connected to MySQL database


# 펀드 ARIMA 분석

In [47]:
# ret = get_arima_predict("K55102B42813",kind="fund")
ret = fetch_prices(cursor,"K55102B42813",kind="fund")
print(ret)
connection.close()

             price
date              
2024-01-02  2000.0
2024-01-03  2000.0
2024-01-04  2000.0
2024-01-05  2000.0
2024-01-08  2000.0
...            ...
2024-06-14  2000.0
2024-06-17  2000.0
2024-06-18  2000.0
2024-06-19  2000.0
2024-06-20  2000.0

[115 rows x 1 columns]


In [33]:
cursor.execute("select fund_code from fund_products_4 where (DATE(arima_update) IS NULL OR DATE(arima_update) <> DATE(NOW()))")
codes = cursor.fetchall()
codes = [ elem[0] for elem in codes]

print(codes)
i = 0

for code in codes:
    print(f"{codes.index(code)+1}/{len(codes)}", code)
    # {'예측값': 14925.268709016453, '예측일': '20240718', '등락률': 5.19}
    result = get_arima_predict(code, kind = "fund")
    predicted_price, percent = result['예측값'], result['등락률']
    # query = f"insert into fund_products_4 (fund_code, arima_price, arima_percent, arima_update) values ('{code}',{predicted_price},{percent},NOW())"
    query = f"""
    UPDATE fund_products_4
    SET arima_price = {predicted_price},
        arima_percent = {percent},
        arima_update = NOW()
    WHERE fund_code = '{code}'
        AND (DATE(arima_update) IS NULL OR DATE(arima_update) <> NOW());
    """
    cursor.execute(query)
    
    if cursor.rowcount == 0:
        print(f"No rows updated for fund code {code}")
    else:
        print(f"Updated {cursor.rowcount} rows for fund code {code}")
        
    connection.commit()


['K55101B79338', 'K55101B79353', 'K55101B79361', 'K55101BA8230', 'K55101BD5035', 'K55101BD5308', 'K55101BI4108', 'K55101BJ5096', 'K55101BP1840', 'K55101BU2759', 'K55101BW9851', 'K55101BW9950', 'K55101BW9968', 'K55101BX0115', 'K55101BX0164', 'K55101BX5403', 'K55101BZ1061', 'K55101BZ1079', 'K55101BZ1087', 'K55101C31949', 'K55101C35296', 'K55101C35304', 'K55101C35312', 'K55101C35338', 'K55101C35346', 'K55101C35379', 'K55101C35387', 'K55101C35395', 'K55101C35403', 'K55101C35429', 'K55101C35437', 'K55101C70822', 'K55101C70830', 'K55101CA2695', 'K55101CF3110', 'K55101D21690', 'K55101D21708', 'K55101D21716', 'K55102B42813', 'K55102BA1655', 'K55102BA2091', 'K55102BA2125', 'K55102BA2141', 'K55102BA4501', 'K55102BB8568', 'K55102BJ4742', 'K55102BM9100', 'K55102BM9118', 'K55102BM9126', 'K55102BM9134', 'K55102BM9142', 'K55102BM9159', 'K55102BN0660', 'K55102BP4975', 'K55102BP4983', 'K55102BW7136', 'K55102BZ4403', 'K55102C00157', 'K55102C00199', 'K55102C00207', 'K55102C00215', 'K55102C00223', 'K55102

In [5]:
def fetch_prices(cursor, code, kind):
    data = []
    if kind == "fund" :
        query = f'select date, price from fund_prices where date between "2024-01-01" and "2024-06-20" and fund_code = "{code}";'
    else :
        query = f'select date, price from stocks_products_details where date between "2024-01-01" and "2024-06-20" and stock_code = "{code}";'
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        data.append(row)
    df = pd.DataFrame(data, columns=['date', 'price'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

In [7]:
# async def insert_data() :
#     query = "select * from stocks_products"
#     cursor.execute(query)
#     row = cursor.fetchone()
#     while row is not None:
#         code = row[0]
#         print(f"code : {code}")
#         returned = await get_arima_predict(code)
#         print(returned)
#         # time.sleep(5)
#         row = cursor.fetchone()
# insert_data()


def return_insert_query(percent, price, code):
    return f"""
    UPDATE stocks_products 
    SET pred_update = now(), 
    predict_precent = {percent}, 
    predict_price = {price}
    WHERE stock_code = {code};
    """

query = "select * from stocks_products"
cursor.execute(query)
rows = cursor.fetchall()  # 모든 결과를 한 번에 가져옴

for row in rows:
    code = row[0]
    print(f"code : {code}")
    # {'예측값': 20243.78831164702, '예측일': '20240711', '등락률': -2.44}
    returned = get_arima_predict(code)
    print(returned)
    if 'error' not in returned:
        query = return_insert_query(returned['등락률'], round(returned['예측값'], 2), code)
        print(query)
    else:
        query = return_insert_query(-100, -100, code)
        print(query)
        
    cursor.execute(query)
    connection.commit()  # 변경 사항을 커밋하여 데이터베이스에 반영


code : 000020


NameError: name 'get_arima_predict' is not defined

In [36]:
def plot(current,predict):
    plt.plot(current, label='종가')
    plt.plot(predict, label='Test')
    plt.legend()
    plt.show()

def get_arima_predict(code,kind):
    try :
        cursor = connection.cursor()
        prices = fetch_prices(cursor,code,kind)
        connection.commit()
        
        model = auto_arima(prices, start_p=1, start_q=1, 
                                    max_p=3, max_1=3, m=3, seasonal=False, # sarima(seasonal=True)
                                    d=1, D=0,
                                    max_P=3, max_Q=3, 
                                    # trace=True,
                                    error_action='ignore',
                                    suppress_warnings=True, 
                                    stepwise=False)
        pred = model.predict(n_periods=30).to_list()

        date_range = pd.date_range(start=prices.index[-1], periods=30, freq='D')
        future_data = pd.DataFrame(data=pred, index=date_range, columns=['Value'])
        plot(prices, future_data)
        result = {'예측값' : future_data.iloc[-1].item(), '예측일' : future_data.index[-1].strftime('%Y%m%d') ,"등락률" : round( (future_data.iloc[-1].item()/prices.iloc[-1].item() -1 )*100,2)}
        return result
        
    except Exception as e:
        return {"error" : "error"}

In [40]:
from transformers import pipeline
from flask import Flask, request, jsonify
import numpy as np
import joblib

# 감정 분석 파이프라인 로드
mapping = {3: '매우공격적', 2: '공격적', 1: '안정적', 0: '소극적'}
map_func = np.vectorize(mapping.get)

app = Flask(__name__)

@app.route('/arima/<code>', methods=['POST'])
def predict(code):
    try:
        arima_result = get_arima_predict(code)
        return jsonify(arima_result)
        
    except Exception as e:
        return jsonify({'error': str(e)})
    
if __name__ == "__main__":
    from waitress import serve
    serve(app, host="0.0.0.0", port=8081)

2024-06-19 04:24:28.715212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 04:24:29.777389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyboardInterrupt: 